## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,San Quintin,MX,30.4833,-115.9500,74.19,64,3,15.21,clear sky
1,1,Nantucket,US,41.2835,-70.0995,77.11,71,99,11.61,overcast clouds
2,2,Punta Arenas,CL,-53.1500,-70.9167,35.71,93,100,3.33,light rain
3,3,Camacha,PT,33.0833,-16.3333,65.62,83,40,17.27,scattered clouds
4,4,Chimbote,PE,-9.0853,-78.5783,67.78,70,3,10.20,clear sky


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip?"))
max_temp = float(input("What is the maximum temperature you would like for your trip?"))

What is the minimum temperature you would like for your trip?75
What is the maximum temperature you would like for your trip?90


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
1,1,Nantucket,US,41.2835,-70.0995,77.11,71,99,11.61,overcast clouds
6,6,Lazaro Cardenas,MX,17.9583,-102.2000,88.88,57,26,5.75,scattered clouds
8,8,Bud,US,39.4470,-86.1758,76.19,97,100,3.29,overcast clouds
9,9,Cuicatlan,MX,17.8000,-96.9667,87.85,51,95,6.44,overcast clouds
10,10,Naze,JP,28.3667,129.4833,77.92,90,100,9.91,overcast clouds
11,11,Tual,ID,-5.6667,132.7500,79.05,79,100,19.10,overcast clouds
12,12,Albany,US,42.6001,-73.9662,84.94,50,100,3.00,overcast clouds
14,14,Salalah,OM,17.0151,54.0924,80.06,84,98,1.99,overcast clouds
15,15,Harper,LR,4.3750,-7.7169,76.84,81,99,11.21,overcast clouds
16,16,Atuona,PF,-9.8000,-139.0333,76.96,81,60,15.61,light rain


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                241
City                   241
Country                241
Lat                    241
Lng                    241
Max Temp               241
Humidity               241
Cloudiness             241
Wind Speed             241
Current Description    241
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Nantucket,US,77.11,overcast clouds,41.2835,-70.0995,
6,Lazaro Cardenas,MX,88.88,scattered clouds,17.9583,-102.2000,
8,Bud,US,76.19,overcast clouds,39.4470,-86.1758,
9,Cuicatlan,MX,87.85,overcast clouds,17.8000,-96.9667,
10,Naze,JP,77.92,overcast clouds,28.3667,129.4833,
11,Tual,ID,79.05,overcast clouds,-5.6667,132.7500,
12,Albany,US,84.94,overcast clouds,42.6001,-73.9662,
14,Salalah,OM,80.06,overcast clouds,17.0151,54.0924,
15,Harper,LR,76.84,overcast clouds,4.3750,-7.7169,
16,Atuona,PF,76.96,light rain,-9.8000,-139.0333,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "hotel",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.


In [13]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df.count()

clean_hotel_df = hotel_df
clean_hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Nantucket,US,77.11,overcast clouds,41.2835,-70.0995,Nantucket
6,Lazaro Cardenas,MX,88.88,scattered clouds,17.9583,-102.2000,Lázaro Cárdenas
8,Bud,US,76.19,overcast clouds,39.4470,-86.1758,Trafalgar
9,Cuicatlan,MX,87.85,overcast clouds,17.8000,-96.9667,San Juan Bautista Cuicatlán
10,Naze,JP,77.92,overcast clouds,28.3667,129.4833,Amami
11,Tual,ID,79.05,overcast clouds,-5.6667,132.7500,Tual
12,Albany,US,84.94,overcast clouds,42.6001,-73.9662,Voorheesville
14,Salalah,OM,80.06,overcast clouds,17.0151,54.0924,Salalah
15,Harper,LR,76.84,overcast clouds,4.3750,-7.7169,Harper
16,Atuona,PF,76.96,light rain,-9.8000,-139.0333,Atuona


In [15]:
# 8a. Create the output File (CSV)
output_data_file = "../Vacation_Search/WeatherPy_vacation.csv"

# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [16]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [20]:
# 11a. Add a marker layer for each city to the map. 
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))